Notebook con la solución del RETO PRÁCTICO 
 ---
**Descripción**
 
Notebook con la solución del RETO PRÁCTICO para la prueba técnica de WOMPI para el cargo Ingeniero de datos

**Fuentes de información**
 
Archivo JSON con información de transacciones
 
- **Archivo Origen:**
   - `transactions_50k.jsonl`
   
 **Versionamiento**
 
 - **1.0** - Creación del Notebook - **Camilo Salazar Ocampo**


# CONFIGURACIÓN

## LIBRERIAS

In [1]:
import pandas as pd
import sys

## PARÁMETROS

In [2]:
input='input/transactions_50k.jsonl'
output = "output/agrupacion_bin_fecha.parquet"

# ETL

## EXTRACCIÓN

In [3]:
try:
    print('leemos el archivo{input}')
    df = pd.read_json(input, lines=True)
    pd.options.display.max_columns = None
except Exception as e:
    print(f"❌ Error inesperado: {e}")
    sys.exit(1)
    
#### encuentro que la columna payment_method_type tiene una anidación de json, entonces normalizamos la columna payment_method_type
df_payment_method_type_expandida = pd.json_normalize(df['payment_method_type'])

# Unimos payment_method_type normalizado al df original
df_normalizado = df.join(df_payment_method_type_expandida.add_prefix('pm_'))

df_normalizado

leemos el archivo{input}


,id,created_at,updated_at,status,payment_method_type,amount_in_cents,pm_type,pm_installments,pm_extra.bin,pm_extra.card_holder,pm_extra.is_three_ds,pm_extra.unique_code,pm_extra.three_ds_auth_type,pm_extra.external_identifier,pm_extra.processor_response_code,pm_extra.authorizer_transaction_id
0,1720000000-77942,2024-07-03 16:12:37,2024-07-03 16:13:29,APPROVED,"{'type': 'CARD', 'extra': {'bin': '450668', 'c...",5155064,CARD,3,450668,Roberto Vargas,True,BFXWRL,01,570405,00,774007591
1,1720000010-45698,2024-04-01 10:40:39,2024-04-01 10:43:49,APPROVED,"{'type': 'CARD', 'extra': {'bin': '512069', 'c...",14700093,CARD,18,512069,Leonardo Acosta,True,LQEZGS,02,171317,00,588734983
2,1720000020-64182,2024-09-11 18:12:59,2024-09-11 18:14:23,APPROVED,"{'type': 'CARD', 'extra': {'bin': '438108', 'c...",2143363,CARD,1,438108,Carolina Leon,True,0OHKPY,02,328223,00,340240645
3,1720000030-25046,2024-09-03 17:15:37,2024-09-03 17:18:16,APPROVED,"{'type': 'CARD', 'extra': {'bin': '459321', 'c...",31278534,CARD,6,459321,Fernando Ortega,False,I66UE0,None,859422,00,528060521
4,1720000040-39508,2024-06-15 10:54:17,2024-06-15 10:57:33,DECLINED,"{'type': 'CARD', 'extra': {'bin': '546637', 'c...",65863594,CARD,12,546637,Jose Aguilar,True,DRX5BM,01,241165,41,107947129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1720499950-76593,2024-08-28 22:47:38,2024-08-28 22:49:51,APPROVED,"{'type': 'CARD', 'extra': {'bin': '411054', 'c...",39422192,CARD,24,411054,Andres Flores,True,GPWSP8,02,406318,00,483977899
49996,1720499960-27306,2024-07-29 00:41:27,2024-07-29 00:43:41,APPROVED,"{'type': 'CARD', 'extra': {'bin': '552558', 'c...",37206925,CARD,6,552558,Juan Perez,False,None,None,850743,00,608624249
49997,1720499970-36071,2024-05-02 16:24:59,2024-05-02 16:27:11,APPROVED,"{'type': 'CARD', 'extra': {'bin': '491617', 'c...",6917657,CARD,3,491617,Miguel Castro,False,LF3AG4,None,151402,00,355248742
49998,1720499980-14044,2024-09-03 13:19:20,2024-09-03 13:22:38,DECLINED,"{'type': 'CARD', 'extra': {'bin': '528209', 'c...",2768095,CARD,12,528209,Andrea Campos,False,NGIA7D,None,481153,05,260116811


## TRANSFORMACIÓN

In [4]:
# Estan solicitando agregar por transacciones aprobadas, por lo tanto debo tomar la fecha de creación de la transacción y filtrar

# Filtramos las transacciones en estado aprobadas
df_aprobadas = df_normalizado[df_normalizado["status"] == "APPROVED"].copy()

#convertimos de datetime a time para poder agrupar por día
df_aprobadas["created_at"] = pd.to_datetime(df_aprobadas["created_at"])
df_aprobadas["fecha"] = df_aprobadas["created_at"].dt.date

df_aprobadas

,id,created_at,updated_at,status,payment_method_type,amount_in_cents,pm_type,pm_installments,pm_extra.bin,pm_extra.card_holder,pm_extra.is_three_ds,pm_extra.unique_code,pm_extra.three_ds_auth_type,pm_extra.external_identifier,pm_extra.processor_response_code,pm_extra.authorizer_transaction_id,fecha
0,1720000000-77942,2024-07-03 16:12:37,2024-07-03 16:13:29,APPROVED,"{'type': 'CARD', 'extra': {'bin': '450668', 'c...",5155064,CARD,3,450668,Roberto Vargas,True,BFXWRL,01,570405,00,774007591,2024-07-03
1,1720000010-45698,2024-04-01 10:40:39,2024-04-01 10:43:49,APPROVED,"{'type': 'CARD', 'extra': {'bin': '512069', 'c...",14700093,CARD,18,512069,Leonardo Acosta,True,LQEZGS,02,171317,00,588734983,2024-04-01
2,1720000020-64182,2024-09-11 18:12:59,2024-09-11 18:14:23,APPROVED,"{'type': 'CARD', 'extra': {'bin': '438108', 'c...",2143363,CARD,1,438108,Carolina Leon,True,0OHKPY,02,328223,00,340240645,2024-09-11
3,1720000030-25046,2024-09-03 17:15:37,2024-09-03 17:18:16,APPROVED,"{'type': 'CARD', 'extra': {'bin': '459321', 'c...",31278534,CARD,6,459321,Fernando Ortega,False,I66UE0,None,859422,00,528060521,2024-09-03
5,1720000050-58805,2024-08-27 19:20:58,2024-08-27 19:21:57,APPROVED,"{'type': 'CARD', 'extra': {'bin': '543421', 'c...",4945806,CARD,24,543421,Jorge Rivera,False,WCHNF2,None,126365,00,347924930,2024-08-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49994,1720499940-81535,2024-09-12 13:18:02,2024-09-12 13:18:53,APPROVED,"{'type': 'CARD', 'extra': {'bin': '483102', 'c...",12510517,CARD,18,483102,Sara Gutierrez,False,None,None,546976,00,868548030,2024-09-12
49995,1720499950-76593,2024-08-28 22:47:38,2024-08-28 22:49:51,APPROVED,"{'type': 'CARD', 'extra': {'bin': '411054', 'c...",39422192,CARD,24,411054,Andres Flores,True,GPWSP8,02,406318,00,483977899,2024-08-28
49996,1720499960-27306,2024-07-29 00:41:27,2024-07-29 00:43:41,APPROVED,"{'type': 'CARD', 'extra': {'bin': '552558', 'c...",37206925,CARD,6,552558,Juan Perez,False,None,None,850743,00,608624249,2024-07-29
49997,1720499970-36071,2024-05-02 16:24:59,2024-05-02 16:27:11,APPROVED,"{'type': 'CARD', 'extra': {'bin': '491617', 'c...",6917657,CARD,3,491617,Miguel Castro,False,LF3AG4,None,151402,00,355248742,2024-05-02


In [5]:
# agrupamos
df_group = (
    df_aprobadas
    .groupby(["pm_extra.bin", "fecha"])
    .agg(
        cantidad_transacciones=("id", "count"),
        monto_aprobado=("amount_in_cents", "sum")
    )
    .reset_index()
)

df_group

,pm_extra.bin,fecha,cantidad_transacciones,monto_aprobado
0,231030,2024-04-01,1,28352749
1,231030,2024-04-02,3,13007751
2,231030,2024-04-03,3,26865765
3,231030,2024-04-04,2,42546032
4,231030,2024-04-05,1,9442522
...,...,...,...,...
21524,555825,2024-09-21,2,13128504
21525,555825,2024-09-22,1,8985614
21526,555825,2024-09-24,2,67417540
21527,555825,2024-09-27,2,15412704


## CARGA

In [ ]:
# Guardamos en parquet
try:
    df_group.to_parquet(output, index=False)
    print("✅ Archivo Parquet guardado exitosamente.")
except Exception as e:
    print(f"❌ Ocurrió un error inesperado al guardar el Parquet: {e}")

✅ Archivo Parquet guardado exitosamente.
